# Face detection using Amazon Rekognition

***
This notebook provides a walkthrough of [face detection API](https://docs.aws.amazon.com/rekognition/latest/dg/faces.html) in Amazon Rekognition to identify faces.
***

In [3]:
# Initialize dependencies
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients
REGION = boto3.session.Session().region_name or 'us-east-1'
rekognition = boto3.client('rekognition', REGION)
s3 = boto3.client('s3', REGION)

bucket_name = 'test-images-891377001208'

# Detect faces in image
***

In [4]:
# Show image
image_name = "looking_at_screen.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to detect faces in the image

<https://docs.aws.amazon.com/rekognition/latest/dg/faces-detect-images.html>

In [5]:
detect_faces_response = rekognition.detect_faces(
    Attributes=['ALL'],
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see faces, detected attributes, confidence score and additional information.

In [6]:
display(detect_faces_response)

{'FaceDetails': [{'BoundingBox': {'Width': 0.1543322205543518,
    'Height': 0.38393434882164,
    'Left': 0.44668450951576233,
    'Top': 0.20054830610752106},
   'AgeRange': {'Low': 30, 'High': 38},
   'Smile': {'Value': False, 'Confidence': 99.94497680664062},
   'Eyeglasses': {'Value': False, 'Confidence': 99.84290313720703},
   'Sunglasses': {'Value': False, 'Confidence': 99.98878479003906},
   'Gender': {'Value': 'Male', 'Confidence': 99.98247528076172},
   'Beard': {'Value': True, 'Confidence': 99.8033447265625},
   'Mustache': {'Value': True, 'Confidence': 64.8669204711914},
   'EyesOpen': {'Value': True, 'Confidence': 97.28457641601562},
   'MouthOpen': {'Value': False, 'Confidence': 99.2823257446289},
   'Emotions': [{'Type': 'CALM', 'Confidence': 83.06396484375},
    {'Type': 'SAD', 'Confidence': 7.379150390625},
    {'Type': 'ANGRY', 'Confidence': 3.6956787109375},
    {'Type': 'CONFUSED', 'Confidence': 1.1240642070770264},
    {'Type': 'DISGUSTED', 'Confidence': 0.08096694

#### Display number of faces detected

In [7]:
print("Number of faces detected: {}".format(len(detect_faces_response['FaceDetails'])))

Number of faces detected: 1


# Recognize faces in video
Face recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/faces-sqs-video.html. 

- First we start a face detection job which returns a Job Id.
- We can then call `get_face_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

#### Show video in the player

In [9]:
video_name = "leaving.mp4"
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


""


#### Call Rekognition to start a job for face detection

In [10]:
start_face_detection = rekognition.start_face_detection(
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

faces_job_id = start_face_detection['JobId']
display("Job Id: {0}".format(faces_job_id))

'Job Id: 0db2ec3f95cc58b2522f563728bfde5a5122bcd30ce8a1d51bb1347575acbe0f'

### Additional (Optional) Request Attributes

ClientRequestToken, JobTag, MinConfidence, and NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html

FaceDetail:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_FaceDetail.html


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [12]:
get_face_detection = rekognition.get_face_detection(
    JobId=faces_job_id
)

while(get_face_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_face_detection = rekognition.get_face_detection(
        JobId=faces_job_id)
    
display(get_face_detection['JobStatus'])

'SUCCEEDED'

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected faces and attributes.
For each detected face, you will see information like Timestamp

In [ ]:
display(get_face_detection)

#### Display face detected by timestamp and alert when faces are not detected

In [13]:
# Faces detected in each frame
prev_ts = 0
threshold = 1000 # ms
for face in get_face_detection['Faces']:
    ts = face["Timestamp"]
    cconfidence = face["Face"]["Confidence"]
    if ts-prev_ts>threshold:
        print("ALERT - no face detected for {} seconds".format((ts-prev_ts)/1000))
    print("Detected face on Timestamp: {} with confidence: {}".format(ts, cconfidence))
    prev_ts = ts

Detected face on Timestamp: 0 with confidence: 99.9998779296875
Detected face on Timestamp: 467 with confidence: 99.99989318847656
Detected face on Timestamp: 967 with confidence: 99.9998550415039
Detected face on Timestamp: 1468 with confidence: 99.99980926513672
Detected face on Timestamp: 1968 with confidence: 99.97500610351562
ALERT - no face detected for 5.506 seconds
Detected face on Timestamp: 7474 with confidence: 99.94216918945312
Detected face on Timestamp: 7974 with confidence: 99.9998779296875


***
### References
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectFaces.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetFaceDetection.html

***

You have successfully used Amazon Rekognition to detect faces in images and videos.